In [ ]:
from backend.llm.llm_factory import LLMFactory
import yaml
from pathlib import Path
from pydantic import BaseModel



class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]


config_path = Path("backend/configs/configs.yml")
with config_path.open("r", encoding="utf-8") as f:
    configs = yaml.safe_load(f) or {}

llm_config = configs.get("LLM", {})
llm_client = LLMFactory.create_client(llm_config)

response = llm_client.generate(prompt="Tell me about Canada.", format=Country)
print(type(response))
print(response)


In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch


model_name = 'NeuML/pubmedbert-base-embeddings'


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

texts = ["This is an example sentence.", "Each sentence is converted into an embedding."]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)


token_embeddings = model_output.last_hidden_state
input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
sentence_embeddings = sum_embeddings / sum_mask


sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)


print("Sentence Embeddings:")
print(sentence_embeddings)
print(f"Shape of embeddings: {sentence_embeddings.shape}")

In [ ]:
from backend.encoders.transformer_encoder import TransformerEncoder

encoder = TransformerEncoder(model_name="NeuML/pubmedbert-base-embeddings", device="cuda")
embs = encoder.embed(["This is a test.", "Another sentence."])

len(embs), len(embs[0])  # embs is a CPU torch.Tensor of shape (2, D)

In [ ]:
from backend.chunkers.semantic_chunking import SemanticChunker
from backend.encoders.transformer_encoder import TransformerEncoder

encoder_model = "intfloat/multilingual-e5-base"
chunker = SemanticChunker(encoder_name="transformer", embedding_model=encoder_model, device="cuda", buffer=0, similarity_threshold=0.9, split_threshold=0.85)
text ="""Tiểu đường (hay còn gọi là đái tháo đường) là một bệnh rối loạn chuyển hóa, đặc trưng bởi lượng đường trong máu luôn cao hơn mức bình thường. Nguyên nhân chính là do cơ thể bị thiếu hụt hoặc đề kháng với insulin, dẫn đến việc không thể chuyển hóa đường thành năng lượng hiệu quả, làm đường tích tụ trong máu. 
Nguyên nhân gây bệnh
Thiếu hụt hoặc đề kháng insulin: Insulin là hormone giúp đưa glucose từ máu vào tế bào để tạo năng lượng. Khi insulin không hoạt động tốt, glucose sẽ tích tụ trong máu.
Di truyền: Yếu tố di truyền cũng có thể làm tăng nguy cơ mắc bệnh.
Lối sống: Chế độ ăn uống không lành mạnh và lười vận động có thể dẫn đến tiểu đường, kể cả ở người gầy. 
Biểu hiện phổ biến
Khát nước và đi tiểu nhiều: Cơ thể cố gắng loại bỏ lượng đường dư thừa ra ngoài qua nước tiểu, gây mất nước và cảm giác khát liên tục.
Sụt cân: Mặc dù ăn nhiều nhưng cơ thể vẫn sụt cân vì không thể sử dụng glucose làm năng lượng.
Mệt mỏi kéo dài: Do cơ thể không có đủ năng lượng để hoạt động.
Nhìn mờ: Lượng đường cao trong máu có thể gây tổn thương các mạch máu nhỏ trong mắt.
Tê bì chân tay: Biến chứng thần kinh có thể dẫn đến cảm giác tê bì.
Vết thương lâu lành: Khả năng tự phục hồi của cơ thể bị suy giảm.

Sốt xuất huyết là bệnh truyền nhiễm cấp tính do virus Dengue gây ra, lây lan chủ yếu qua vết đốt của muỗi vằn mang virus. Bệnh có thể có nhiều triệu chứng, từ nhẹ đến nặng, và nguy hiểm nhất là có thể dẫn đến sốc, rối loạn đông máu, suy tạng và tử vong nếu không được điều trị kịp thời. Hiện chưa có thuốc đặc trị và vắc xin phòng bệnh, nên biện pháp phòng ngừa hiệu quả là diệt muỗi, lăng quăng, bọ gậy và phòng tránh muỗi đốt.  
Đặc điểm của bệnh
Nguyên nhân: Virus Dengue, với 4 tuýp huyết thanh DENV-1, DENV-2, DENV-3, DENV-4. 
Cách lây truyền: Qua trung gian là muỗi vằn cái, chủ yếu là loài Aedes aegypti. 
Biểu hiện: Bệnh có thể biểu hiện đa dạng, thường khởi phát đột ngột với sốt cao (40°C), đau đầu, đau hốc mắt, đau cơ, khớp, buồn nôn, nôn, phát ban và sưng hạch bạch huyết. 
Giai đoạn bệnh: Bệnh thường trải qua 3 giai đoạn: giai đoạn sốt, giai đoạn nguy hiểm và giai đoạn hồi phục. 
Biến chứng nguy hiểm: Khi bệnh chuyển sang giai đoạn nguy hiểm có thể gây sốc, chảy máu trong, rối loạn đông máu, suy tạng, trụy tim mạch và dẫn đến tử vong. 
Cách phòng bệnh
Chủ động diệt muỗi, diệt lăng quăng và bọ gậy trong và quanh nhà. 
Tránh để muỗi đốt bằng cách mặc quần áo dài tay, sử dụng màn khi ngủ, và dùng thuốc chống muỗi. 
Phát hiện sớm các triệu chứng và đến cơ sở y tế để được chẩn đoán và điều trị kịp thời. 
"""

chunks = chunker.chunk(text)
print(f"Generated {len(chunks)} chunks:\n")
for i, c in enumerate(chunks, 1):
    print(f"--- Chunk {i} ---")
    print(c)

In [ ]:
import re
import numpy as np
from backend.encoders.transformer_encoder import TransformerEncoder
encoder = chunker._get_encoder()
sentences = [s.strip() for s in re.split(r'(?<=[\.\?!])\s+', text.strip()) if s.strip()]
embs = encoder.embed(sentences, batch_size=chunker.batch_size)
embs_np = embs.numpy()
def cosine(a, b):
    an = np.linalg.norm(a)
    bn = np.linalg.norm(b)
    if an == 0 or bn == 0:
        return 0.0
    return float(np.dot(a, b) / (an * bn))
print(f"Total sentences: {len(sentences)}")
for i in range(len(sentences)-1):
    sim = cosine(embs_np[i], embs_np[i+1])
    prefix = sentences[i][:120].replace('\n',' ')
    suffix = sentences[i+1][:120].replace('\n',' ')
    print(f"sim[{i}->{i+1}] = {sim:.4f}  --  {prefix!r}  >>>  {suffix!r}")
boundary_idx = next((i for i,s in enumerate(sentences) if 'Sốt xuất huyết' in s or 'Sốt xuất huyết' in s.replace('\n',' ')), None)
if boundary_idx is not None and boundary_idx>0:
    bsim = cosine(embs_np[boundary_idx-1], embs_np[boundary_idx])
    print('\nDetected paragraph-start sentence index:', boundary_idx)
    print(f'Similarity across paragraph boundary (sentence {boundary_idx-1} -> {boundary_idx}): {bsim:.4f}')
else:
    print('\nCould not detect paragraph boundary sentence by keyword; inspect printed sentences to find it.')

Hãy tìm trên internet các công trình nghiên cứu về việc sử dụng LLM để trích xuất entity từ văn bản y tế.
Sau đó hãy viết class node extractor sử dụng các tài nguyên có sẵn như "backend/llm/ollama_client.py", "backend/encoders/transformer_encoder.py" để trích xuất các entity

Input:
- Văn bản y tế (đoạn văn ngắn về lĩnh vực y tế)
- LLM client: client llm được chỉ định (ollama, gemini,...)
- Model name: model name ứng với client
- Embedding model

Output:
Là một list các entity được trích xuất, mỗi entity bao gồm
- name: tên của entity
- mention: phần văn bản chứa context của entity
- name_embedding: embedding của name
- mention_embedding: ebedding của văn bản mention

Yêu cầu:
- Hoàn thành cài đặt cho class node extraction 
- Nghiên cứu các SOTA cùng topic lựa chọn các prompt tối ưu nhất và áp dụng vào node extraction
- Luôn sử dụng schema (structured output) để lấy dữ liệu output là JSON
- Viết file main.py kết hợp các bước xử lý theo luồng sau:
    1. Load data path từ "data/500_samples_pmc" (giới hạn n sample cho quá trình test)
    2. Load config từ "backend/configs/configs.yml"
    3. Chunking sử dụng section chunking
    4. Trích xuất node

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("dyyyyyyyy/GNER-T5-xxl")
model = AutoModelForSeq2SeqLM.from_pretrained("dyyyyyyyy/GNER-T5-xxl", torch_dtype=torch.bfloat16).cuda()
model = model.eval()
instruction_template = "Please analyze the sentence provided, identifying the type of entity for each word on a token-by-token basis.\nOutput format is: word_1(label_1), word_2(label_2), ...\nWe'll use the BIO-format to label the entities, where:\n1. B- (Begin) indicates the start of a named entity.\n2. I- (Inside) is used for words within a named entity but are not the first word.\n3. O (Outside) denotes words that are not part of a named entity.\n"
sentence = "did george clooney make a musical in the 1980s"
entity_labels = ["genre", "rating", "review", "plot", "song", "average ratings", "director", "character", "trailer", "year", "actor", "title"]
instruction = f"{instruction_template}\nUse the specific entity tags: {', '.join(entity_labels)} and O.\nSentence: {sentence}"
inputs = tokenizer(instruction, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=640)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


## UMLS EDA

In [ ]:
# Test de_node_extractor on provided medical text
from pathlib import Path
import json
import importlib
import sys

# Reload module to pick up changes
if 'backend.node_extractor.de_node_extractor' in sys.modules:
    import backend.node_extractor.de_node_extractor
    importlib.reload(backend.node_extractor.de_node_extractor)

# Prepare the test text
text = '''Tiểu đường (hay còn gọi là đái tháo đường) là một bệnh rối loạn chuyển hóa, đặc trưng bởi lượng đường trong máu luôn cao hơn mức bình thường. Nguyên nhân chính là do cơ thể bị thiếu hụt hoặc đề kháng với insulin, dẫn đến việc không thể chuyển hóa đường thành năng lượng hiệu quả, làm đường tích tụ trong máu. 
Nguyên nhân gây bệnh
Thiếu hụt hoặc đề kháng insulin: Insulin là hormone giúp đưa glucose từ máu vào tế bào để tạo năng lượng. Khi insulin không hoạt động tốt, glucose sẽ tích tụ trong máu.
Di truyền: Yếu tố di truyền cũng có thể làm tăng nguy cơ mắc bệnh.
Lối sống: Chế độ ăn uống không lành mạnh và lười vận động có thể dẫn đến tiểu đường, kể cả ở người gầy. 
Biểu hiện phổ biến
Khát nước và đi tiểu nhiều: Cơ thể cố gắng loại bỏ lượng đường dư thừa ra ngoài qua nước tiểu, gây mất nước và cảm giác khát liên tục.
Sụt cân: Mặc dù ăn nhiều nhưng cơ thể vẫn sụt cân vì không thể sử dụng glucose làm năng lượng.
Mệt mỏi kéo dài: Do cơ thể không có đủ năng lượng để hoạt động.
Nhìn mờ: Lượng đường cao trong máu có thể gây tổn thương các mạch máu nhỏ trong mắt.
Tê bì chân tay: Biến chứng thần kinh có thể dẫn đến cảm giác tê bì.
Vết thương lâu lành: Khả năng tự phục hồi của cơ thể bị suy giảm.

Sốt xuất huyết là bệnh truyền nhiễm cấp tính do virus Dengue gây ra, lây lan chủ yếu qua vết đốt của muỗi vằn mang virus. Bệnh có thể có nhiều triệu chứng, từ nhẹ đến nặng, và nguy hiểm nhất là có thể dẫn đến sốc, rối loạn đông máu, suy tạng và tử vong nếu không được điều trị kịp thời. Hiện chưa có thuốc đặc trị và vắc xin phòng bệnh, nên biện pháp phòng ngừa hiệu quả là diệt muỗi, lăng quăng, bọ gậy và phòng tránh muỗi đốt.  
Đặc điểm của bệnh
Nguyên nhân: Virus Dengue, với 4 tuýp huyết thanh DENV-1, DENV-2, DENV-3, DENV-4. 
Cách lây truyền: Qua trung gian là muỗi vằn cái, chủ yếu là loài Aedes aegypti. 
Biểu hiện: Bệnh có thể biểu hiện đa dạng, thường khởi phát đột ngột với sốt cao (40°C), đau đầu, đau hốc mắt, đau cơ, khớp, buồn nôn, nôn, phát ban và sưng hạch bạch huyết. 
Giai đoạn bệnh: Bệnh thường trải qua 3 giai đoạn: giai đoạn sốt, giai đoạn nguy hiểm và giai đoạn hồi phục. 
Biến chứng nguy hiểm: Khi bệnh chuyển sang giai đoạn nguy hiểm có thể gây sốc, chảy máu trong, rối loạn đông máu, suy tạng, trụy tim mạch và dẫn đến tử vong. 
Cách phòng bệnh
Chủ động diệt muỗi, diệt lăng quăng và bọ gậy trong và quanh nhà. 
Tránh để muỗi đốt bằng cách mặc quần áo dài tay, sử dụng màn khi ngủ, và dùng thuốc chống muỗi. 
Phát hiện sớm các triệu chứng và đến cơ sở y tế để được chẩn đoán và điều trị kịp thời. 
'''

# Ensure we have an llm_client and configs (cells above create them; if not, create minimal client)
try:
    llm_client  # defined in an earlier cell
except NameError:
    from backend.llm.llm_factory import LLMFactory
    import yaml
    cfg_path = Path('backend/configs/configs.yml')
    with cfg_path.open('r', encoding='utf-8') as f:
        configs = yaml.safe_load(f) or {}
    llm_config = configs.get('LLM', {})
    llm_client = LLMFactory.create_client(llm_config)
    # keep llm_config in scope
    try:
        globals()['llm_config'] = llm_config
    except Exception:
        pass

# Determine model and embedding model names
model_name = globals().get('llm_config', {}).get('model_name', 'gpt-like-model')
embedding_model = globals().get('configs', {}).get('EMBEDDING', {}).get('model', 'intfloat/multilingual-e5-base')

# Import and run the de_node_extractor
from backend.node_extractor.de_node_extractor import NodeExtractor as DE_NodeExtractor
from backend.encoders.transformer_encoder import TransformerEncoder

# Instantiate extractor (this will create an encoder if not provided)
extractor = DE_NodeExtractor(llm_client=llm_client, model_name=model_name, embedding_model=embedding_model, device='cpu')

print('Running de_node_extractor.extract(...) — this will call the LLM up to 5 times (one per super-prompt)')
nodes = extractor.extract(text)
print(f'\nExtracted {len(nodes)} nodes')
# Print first 25 nodes for inspection
for i, n in enumerate(nodes, start=1):
    print(f"{i:02d}. name: {n.get('name')!r}  | semantic_type: {n.get('semantic_type')!r}")

# Save results to output directory for later inspection
out_dir = Path('output/entities_extracted_2')
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / 'de_node_extractor_test_nodes.json'
with out_path.open('w', encoding='utf-8') as f:
    json.dump(nodes, f, ensure_ascii=False, indent=2)
print('\nSaved nodes to', out_path)

## Test 15 Super Prompts (UMLS Semantic Groups)

In [ ]:
# Test de_node_extractor with NEW 15 prompts
from pathlib import Path
import json
import importlib
import sys

# Reload modules to pick up changes
for mod in ['backend.node_extractor.supper_prompts', 'backend.node_extractor.schema', 'backend.node_extractor.de_node_extractor']:
    if mod in sys.modules:
        importlib.reload(sys.modules[mod])

# Short test text
test_text = '''Bệnh tiểu đường là một rối loạn chuyển hóa do thiếu insulin hoặc kháng insulin. 
Triệu chứng bao gồm khát nước nhiều, đi tiểu thường xuyên, và mệt mỏi. 
Điều trị bao gồm tiêm insulin và kiểm soát chế độ ăn.'''

# Get LLM client (should be available from previous cells)
try:
    llm_client
    print("✓ LLM client available")
except NameError:
    from backend.llm.llm_factory import LLMFactory
    import yaml
    cfg_path = Path('backend/configs/configs.yml')
    with cfg_path.open('r', encoding='utf-8') as f:
        configs = yaml.safe_load(f) or {}
    llm_config = configs.get('LLM', {})
    llm_client = LLMFactory.create_client(llm_config)
    model_name = llm_config.get('model_name', 'gpt-like-model')
    embedding_model = configs.get('EMBEDDING', {}).get('model', 'intfloat/multilingual-e5-base')
    print("✓ Created new LLM client")

# Import the updated NodeExtractor
from backend.node_extractor.de_node_extractor import NodeExtractor as DE_NodeExtractor

# Create extractor instance
extractor_15 = DE_NodeExtractor(
    llm_client=llm_client, 
    model_name=model_name, 
    embedding_model=embedding_model, 
    device='cpu'
)

print(f"\n{'='*80}")
print("Testing with 15 UMLS Semantic Group Prompts")
print(f"{'='*80}\n")
print(f"Input text ({len(test_text)} chars):")
print(f"{test_text[:150]}...\n")

# Extract entities
print("Running extraction (will call LLM 15 times)...")
nodes_15 = extractor_15.extract(test_text)

print(f"\n{'='*80}")
print(f"✓ Extracted {len(nodes_15)} unique entities")
print(f"{'='*80}\n")

# Display results
print("Results:")
for i, node in enumerate(nodes_15[:30], start=1):
    name = node.get('name')
    sem_type = node.get('semantic_type', '')
    has_emb = '✓' if node.get('name_embedding') else '✗'
    print(f"{i:02d}. [{has_emb}] {name:40s} | {sem_type}")

if len(nodes_15) > 30:
    print(f"\n... and {len(nodes_15) - 30} more entities")

# Save results
out_dir = Path('output/entities_extracted_2')
out_dir.mkdir(parents=True, exist_ok=True)
out_path_15 = out_dir / 'test_15_prompts_nodes.json'
with out_path_15.open('w', encoding='utf-8') as f:
    json.dump(nodes_15, f, ensure_ascii=False, indent=2)

print(f"\n✓ Saved results to: {out_path_15}")

## Test 15 Prompts + Filter Stage (2-Stage Pipeline)

In [ ]:
# Test de_node_extractor with 2-STAGE PIPELINE: 15 prompts + Filter
from pathlib import Path
import json
import importlib
import sys

# Reload modules to pick up changes
for mod in ['backend.node_extractor.supper_prompts', 'backend.node_extractor.de_node_extractor']:
    if mod in sys.modules:
        importlib.reload(sys.modules[mod])

# Test text
test_text = '''Tiểu đường (hay còn gọi là đái tháo đường) là một bệnh rối loạn chuyển hóa, đặc trưng bởi lượng đường trong máu luôn cao hơn mức bình thường. Nguyên nhân chính là do cơ thể bị thiếu hụt hoặc đề kháng với insulin, dẫn đến việc không thể chuyển hóa đường thành năng lượng hiệu quả, làm đường tích tụ trong máu. 
Nguyên nhân gây bệnh
Thiếu hụt hoặc đề kháng insulin: Insulin là hormone giúp đưa glucose từ máu vào tế bào để tạo năng lượng. Khi insulin không hoạt động tốt, glucose sẽ tích tụ trong máu.
Di truyền: Yếu tố di truyền cũng có thể làm tăng nguy cơ mắc bệnh.
Lối sống: Chế độ ăn uống không lành mạnh và lười vận động có thể dẫn đến tiểu đường, kể cả ở người gầy. 
Biểu hiện phổ biến
Khát nước và đi tiểu nhiều: Cơ thể cố gắng loại bỏ lượng đường dư thừa ra ngoài qua nước tiểu, gây mất nước và cảm giác khát liên tục.
Sụt cân: Mặc dù ăn nhiều nhưng cơ thể vẫn sụt cân vì không thể sử dụng glucose làm năng lượng.
Mệt mỏi kéo dài: Do cơ thể không có đủ năng lượng để hoạt động.
Nhìn mờ: Lượng đường cao trong máu có thể gây tổn thương các mạch máu nhỏ trong mắt.
Tê bì chân tay: Biến chứng thần kinh có thể dẫn đến cảm giác tê bì.
Vết thương lâu lành: Khả năng tự phục hồi của cơ thể bị suy giảm.

Sốt xuất huyết là bệnh truyền nhiễm cấp tính do virus Dengue gây ra, lây lan chủ yếu qua vết đốt của muỗi vằn mang virus. Bệnh có thể có nhiều triệu chứng, từ nhẹ đến nặng, và nguy hiểm nhất là có thể dẫn đến sốc, rối loạn đông máu, suy tạng và tử vong nếu không được điều trị kịp thời. Hiện chưa có thuốc đặc trị và vắc xin phòng bệnh, nên biện pháp phòng ngừa hiệu quả là diệt muỗi, lăng quăng, bọ gậy và phòng tránh muỗi đốt.  
Đặc điểm của bệnh
Nguyên nhân: Virus Dengue, với 4 tuýp huyết thanh DENV-1, DENV-2, DENV-3, DENV-4. 
Cách lây truyền: Qua trung gian là muỗi vằn cái, chủ yếu là loài Aedes aegypti. 
Biểu hiện: Bệnh có thể biểu hiện đa dạng, thường khởi phát đột ngột với sốt cao (40°C), đau đầu, đau hốc mắt, đau cơ, khớp, buồn nôn, nôn, phát ban và sưng hạch bạch huyết. 
Giai đoạn bệnh: Bệnh thường trải qua 3 giai đoạn: giai đoạn sốt, giai đoạn nguy hiểm và giai đoạn hồi phục. 
Biến chứng nguy hiểm: Khi bệnh chuyển sang giai đoạn nguy hiểm có thể gây sốc, chảy máu trong, rối loạn đông máu, suy tạng, trụy tim mạch và dẫn đến tử vong. 
Cách phòng bệnh
Chủ động diệt muỗi, diệt lăng quăng và bọ gậy trong và quanh nhà. 
Tránh để muỗi đốt bằng cách mặc quần áo dài tay, sử dụng màn khi ngủ, và dùng thuốc chống muỗi. 
Phát hiện sớm các triệu chứng và đến cơ sở y tế để được chẩn đoán và điều trị kịp thời. 
'''

# Get LLM client
try:
    llm_client
    print("✓ LLM client available")
except NameError:
    from backend.llm.llm_factory import LLMFactory
    import yaml
    cfg_path = Path('backend/configs/configs.yml')
    with cfg_path.open('r', encoding='utf-8') as f:
        configs = yaml.safe_load(f) or {}
    llm_config = configs.get('LLM', {})
    llm_client = LLMFactory.create_client(llm_config)
    model_name = llm_config.get('model_name', 'gpt-like-model')
    embedding_model = configs.get('EMBEDDING', {}).get('model', 'intfloat/multilingual-e5-base')
    print("✓ Created new LLM client")

# Import the updated NodeExtractor
from backend.node_extractor.de_node_extractor import NodeExtractor as DE_NodeExtractor

print(f"\n{'='*80}")
print("TEST 1: WITHOUT Filter (15 prompts only - expect issues)")
print(f"{'='*80}\n")

# Create extractor WITHOUT filter
extractor_no_filter = DE_NodeExtractor(
    llm_client=llm_client, 
    model_name=model_name, 
    embedding_model=embedding_model, 
    device='cpu',
    use_filter=False  # Disable filter
)

print("Running extraction WITHOUT filter (15 LLM calls)...")
nodes_no_filter = extractor_no_filter.extract(test_text)
print(f"✓ Extracted {len(nodes_no_filter)} entities (raw, unfiltered)\n")

# Display first 20
print("First 20 results (NO FILTER):")
for i, node in enumerate(nodes_no_filter[:20], start=1):
    name = node.get('name')
    sem_type = node.get('semantic_type', '')
    print(f"{i:02d}. {name:40s} | {sem_type}")

print(f"\n{'='*80}")
print("TEST 2: WITH Filter (15 prompts + 1 filter = 16 LLM calls)")
print(f"{'='*80}\n")

# Create extractor WITH filter
extractor_with_filter = DE_NodeExtractor(
    llm_client=llm_client, 
    model_name=model_name, 
    embedding_model=embedding_model, 
    device='cpu',
    use_filter=True  # Enable filter (default)
)

print("Running extraction WITH filter (15 extraction + 1 filter LLM call)...")
nodes_with_filter = extractor_with_filter.extract(test_text)
print(f"✓ Extracted {len(nodes_with_filter)} entities (filtered & consolidated)\n")

# Display all results
print("All results (WITH FILTER):")
for i, node in enumerate(nodes_with_filter, start=1):
    name = node.get('name')
    sem_type = node.get('semantic_type', '')
    has_emb = '✓' if node.get('name_embedding') else '✗'
    print(f"{i:02d}. [{has_emb}] {name:40s} | {sem_type}")

# Save both results for comparison
out_dir = Path('output/entities_extracted_2')
out_dir.mkdir(parents=True, exist_ok=True)

out_no_filter = out_dir / 'test_no_filter_nodes.json'
with out_no_filter.open('w', encoding='utf-8') as f:
    json.dump(nodes_no_filter, f, ensure_ascii=False, indent=2)

out_with_filter = out_dir / 'test_with_filter_nodes.json'
with out_with_filter.open('w', encoding='utf-8') as f:
    json.dump(nodes_with_filter, f, ensure_ascii=False, indent=2)

print(f"\n{'='*80}")
print("COMPARISON:")
print(f"  Without filter: {len(nodes_no_filter)} entities (contains duplicates, hallucinations, misclassifications)")
print(f"  With filter:    {len(nodes_with_filter)} entities (cleaned, validated, consolidated)")
print(f"  Reduction:      {len(nodes_no_filter) - len(nodes_with_filter)} entities removed by filter")
print(f"{'='*80}")

print(f"\n✓ Saved results:")
print(f"  - No filter:   {out_no_filter}")
print(f"  - With filter: {out_with_filter}")

In [ ]:
# DEBUG: Test extraction with debug mode to see what's happening
from pathlib import Path
import json
import importlib
import sys

# Reload modules to pick up latest changes
for mod in ['backend.node_extractor.supper_prompts', 'backend.node_extractor.de_node_extractor']:
    if mod in sys.modules:
        importlib.reload(sys.modules[mod])

# Short test text
test_text = '''Tiểu đường là một bệnh do thiếu insulin. Triệu chứng bao gồm khát nước, đi tiểu thường xuyên, và mệt mỏi. Virus Dengue gây bệnh sốt xuất huyết.'''

# Get LLM client
try:
    llm_client
    print("✓ LLM client available")
except NameError:
    from backend.llm.llm_factory import LLMFactory
    import yaml
    cfg_path = Path('backend/configs/configs.yml')
    with cfg_path.open('r', encoding='utf-8') as f:
        configs = yaml.safe_load(f) or {}
    llm_config = configs.get('LLM', {})
    llm_client = LLMFactory.create_client(llm_config)
    model_name = llm_config.get('model_name', 'gpt-like-model')
    embedding_model = configs.get('EMBEDDING', {}).get('model', 'intfloat/multilingual-e5-base')
    print("✓ Created new LLM client")

# Import the updated NodeExtractor
from backend.node_extractor.de_node_extractor import NodeExtractor as DE_NodeExtractor

print(f"\n{'='*80}")
print("DEBUG TEST: Entity Extraction with Debug Mode")
print(f"{'='*80}\n")

# Create extractor WITH debug mode
extractor = DE_NodeExtractor(
    llm_client=llm_client, 
    model_name=model_name, 
    embedding_model=embedding_model, 
    device='cpu',
    use_filter=False  # Disable filter for cleaner debug output
)

print(f"Test text ({len(test_text)} chars):\n{test_text}\n")
print(f"{'='*80}\n")

# Extract with debug
nodes = extractor.extract(test_text, debug=True)

print(f"\n{'='*80}")
print(f"✓ Final result: {len(nodes)} entities extracted")
print(f"{'='*80}\n")

for i, node in enumerate(nodes, start=1):
    name = node.get('name')
    sem_type = node.get('semantic_type', '')
    print(f"{i:02d}. {name:40s} | {sem_type}")


In [ ]:
import json
from pathlib import Path
data = json.loads(Path('output/umls/clusters/T052_Activity.json').read_text())
print(data['nodes'])

In [ ]:
from graphviz import Digraph

data = [
    {'tui': 'T052', 'sty': 'Activity', 'depth': 1},
    {'tui': 'T053', 'sty': 'Behavior', 'depth': 2},
    {'tui': 'T054', 'sty': 'Social Behavior', 'depth': 3},
    {'tui': 'T055', 'sty': 'Individual Behavior', 'depth': 3},
    {'tui': 'T056', 'sty': 'Daily or Recreational Activity', 'depth': 2},
    {'tui': 'T057', 'sty': 'Occupational Activity', 'depth': 2},
    {'tui': 'T058', 'sty': 'Health Care Activity', 'depth': 3},
    {'tui': 'T059', 'sty': 'Laboratory Procedure', 'depth': 4},
    {'tui': 'T060', 'sty': 'Diagnostic Procedure', 'depth': 4},
    {'tui': 'T061', 'sty': 'Therapeutic or Preventive Procedure', 'depth': 4},
    {'tui': 'T062', 'sty': 'Research Activity', 'depth': 3},
    {'tui': 'T063', 'sty': 'Molecular Biology Research Technique', 'depth': 4},
    {'tui': 'T064', 'sty': 'Governmental or Regulatory Activity', 'depth': 3},
    {'tui': 'T065', 'sty': 'Educational Activity', 'depth': 3},
    {'tui': 'T066', 'sty': 'Machine Activity', 'depth': 2},
]

# Tạo graph
dot = Digraph(comment="UMLS Semantic Type Hierarchy")
dot.attr(rankdir="TB", fontsize="12")

# Nhóm theo depth
levels = {}
for item in data:
    levels.setdefault(item["depth"], []).append(item)

# Sắp xếp lại theo depth
sorted_depths = sorted(levels.keys())

# Thêm node
for depth in sorted_depths:
    for item in levels[depth]:
        node_id = item["tui"]
        label = f"{item['sty']} ({item['tui']})"
        dot.node(node_id, label)

# Tạo cạnh: node depth n → node depth n+1
for i in range(len(sorted_depths)-1):
    d1, d2 = sorted_depths[i], sorted_depths[i+1]
    for parent in levels[d1]:
        for child in levels[d2]:
            dot.edge(parent["tui"], child["tui"])

# Xuất file
dot.render("semantic_type_hierarchy.gv", view=True)
print("Done! File semantic_type_hierarchy.gv created.") 


In [1]:
from backend.node_extractor.de_node_extractor import NodeExtractor
from backend.llm.llm_factory import LLMFactory
import yaml
from pathlib import Path

config_path = Path("backend/configs/configs.yml")
with config_path.open("r", encoding="utf-8") as f:
    configs = yaml.safe_load(f) or {}
llm_config = configs.get("LLM", {})
llm_client = LLMFactory.create_client(llm_config)
extractor = NodeExtractor(llm_client=llm_client, model_name=llm_config.get("model_name", "gpt-like-model"), embedding_model="intfloat/multilingual-e5-base", device="cpu")


sample_text = """
    The patient was diagnosed with Escherichia coli infection after urine culture. 
    MRI showed a lesion in the left temporal lobe. 
    Blood tests revealed elevated levels of C-reactive protein and hemoglobin A1c. 
    The patient takes metformin and lisinopril daily. 
    He has a family history of type 2 diabetes mellitus. 
    His occupation is software engineer and he lives in Boston. 
    Physical examination showed mild edema in the lower extremities.
    """

res = extractor.extract(sample_text)


/home/hung/miniconda3/envs/e1/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracted Entities: {'activity': {'Activity': ['physical therapy'], 'Occupational_Activity': ['software engineer'], 'Health_Care_Activity': ['urine culture', 'MRI', 'blood tests', 'physical examination'], 'Laboratory_Procedure': ['urine culture', 'MRI', 'blood tests'], 'Diagnostic_Procedure': ['urine culture', 'MRI']}, 'phenomenon': {'Phenomenon_or_Process': ['infection'], 'Physiologic_Function': ['elevated levels'], 'Organism_Function': ['infection'], 'Pathologic_Function': ['lesion', 'mild edema'], 'Disease_or_Syndrome': ['Escherichia coli infection', 'type 2 diabetes mellitus']}, 'physical_object': {'Physical_Object': ['lesion', 'temporal lobe'], 'Organism': ['Escherichia coli'], 'Bacterium': ['Escherichia coli'], 'Human': ['patient'], 'Anatomical_Structure': ['temporal lobe', 'lower extremities'], 'Fully_Formed_Anatomical_Structure': ['temporal lobe', 'lower extremities'], 'Body_Part_Organ_or_Organ_Component': ['temporal lobe', 'lower extremities'], 'Anatomical_Abnormality': ['lesi

In [ ]:
#Extracted Entities : 
{
    'activity': {
        'Activity': ['physical therapy'],
        'Occupational_Activity': ['software engineer'],
        'Health_Care_Activity': [
            'urine culture', 'MRI', 'blood tests', 'physical examination'
        ],
        'Laboratory_Procedure': [
            'urine culture', 'MRI', 'blood tests'
        ],
        'Diagnostic_Procedure': ['urine culture', 'MRI']
    },
    'phenomenon': {
        'Phenomenon_or_Process': ['infection'],
        'Physiologic_Function': ['elevated levels'],
        'Organism_Function': ['infection'],
        'Pathologic_Function': [
            'lesion', 'mild edema'
        ],
        'Disease_or_Syndrome': ['Escherichia coli infection', 'type 2 diabetes mellitus']
    },
    'physical_object': {
        'Physical_Object': [
            'lesion', 'temporal lobe'
        ],
        'Organism': ['Escherichia coli'],
        'Bacterium': ['Escherichia coli'],
        'Human': ['patient'],
        'Anatomical_Structure': [
            'temporal lobe', 'lower extremities'
        ],
        'Fully_Formed_Anatomical_Structure': [
            'temporal lobe', 'lower extremities'
        ],
        'Body_Part_Organ_or_Organ_Component': [
            'temporal lobe', 'lower extremities'
        ],
        'Anatomical_Abnormality': ['lesion'],
        'Clinical_Drug': [
            'metformin', 'lisinopril'
        ],
        'Substance': [
            'C-reactive protein', 'hemoglobin A1c'
        ],
        'Body_Substance': ['urine'],
        'Pharmacologic_Substance': [
            'metformin', 'lisinopril'
        ],
        'Biologically_Active_Substance': ['C-reactive protein', 'hemoglobin A1c']
    },
    'conceptual_entity': {
        'Organism_Attribute': ['Escherichia coli'],
        'Clinical_Attribute': [
            'elevated levels of C-reactive protein', 'hemoglobin A1c'
        ],
        'Finding': [
            'lesion in the left temporal lobe', 'mild edema in the lower extremities'
        ],
        'Sign_or_Symptom': ['edema'],
        'Body_Location_or_Region': [
            'left temporal lobe', 'lower extremities'
        ],
        'Geographic_Area': ['Boston'],
        'Occupation_or_Discipline': ['software engineer'],
        'Family_Group': ['type 2 diabetes mellitus']
    }
}
#Filtered Entities : 
{
    'activity': {
        'Activity': ['physical therapy'],
        'Occupational_Activity': ['software engineer'],
        'Laboratory_Procedure': [
            'urine culture', 'MRI', 'blood tests'
        ],
        'Diagnostic_Procedure': [
            'urine culture', 'MRI'
        ],
        'Health_Care_Activity': ['physical examination']
    },
    'phenomenon': {
        'Organism_Function': ['infection'],
        'Physiologic_Function': ['elevated levels'],
        'Pathologic_Function': ['mild edema'],
        'Disease_or_Syndrome': ['Escherichia coli infection', 'type 2 diabetes mellitus']
    },
    'physical_object': {
        'Anatomical_Abnormality': ['lesion'],
        'Body_Part_Organ_or_Organ_Component': [
            'temporal lobe', 'lower extremities'
        ],
        'Bacterium': ['Escherichia coli'],
        'Human': ['patient'],
        'Pharmacologic_Substance': [
            'metformin', 'lisinopril'
        ],
        'Biologically_Active_Substance': [
            'C-reactive protein', 'hemoglobin A1c'
        ],
        'Body_Substance': ['urine']
    },
    'conceptual_entity': {
        'Occupation_or_Discipline': ['software engineer'],
        'Family_Group': ['type 2 diabetes mellitus'],
        'Body_Location_or_Region': [
            'lower extremities', 'left temporal lobe'
        ],
        'Clinical_Attribute': ['elevated levels of C-reactive protein'],
        'Finding': [
            'lesion in the left temporal lobe', 'mild edema in the lower extremities'
        ],
        'Sign_or_Symptom': ['edema'],
        'Geographic_Area': ['Boston']
    }
}
#LLM Filtered Entities : 
{
    'entities': [
        Entity(
            name = 'Escherichia coli infection',
            semantic_type = 'Disease_or_Syndrome',
            mention = ''
        ),
        Entity(name = 'MRI', semantic_type = 'Diagnostic_Procedure'),
        Entity(name = 'lesion in the left temporal lobe', semantic_type = 'Finding'),
        Entity(
            name = 'C-reactive protein',
            semantic_type = 'Biologically_Active_Substance'
        ),
        Entity(
            name = 'hemoglobin A1c',
            semantic_type = 'Biologically_Active_Substance'
        ),
        Entity(name = 'metformin', semantic_type = 'Pharmacologic_Substance'),
        Entity(name = 'lisinopril', semantic_type = 'Pharmacologic_Substance'),
        Entity(
            name = 'type 2 diabetes mellitus',
            semantic_type = 'Disease_or_Syndrome'
        ),
        Entity(name = 'physical examination', semantic_type = 'Health_Care_Activity')
    ]
}

In [2]:
res 

[{'name': 'Escherichia coli infection',
  'semantic_type': 'Disease_or_Syndrome',
  'embedding': [0.01182117685675621,
   0.05496746301651001,
   -0.01459168829023838,
   0.04377296194434166,
   0.04181868955492973,
   -0.034756578505039215,
   -0.026715131476521492,
   -0.03434497117996216,
   0.028154432773590088,
   -0.005912471562623978,
   0.00804898515343666,
   0.01920195482671261,
   0.09786458313465118,
   0.03808759152889252,
   -0.04265206307172775,
   -0.05026828497648239,
   0.015716219320893288,
   -0.008939473889768124,
   0.031530074775218964,
   0.014934376813471317,
   0.030743354931473732,
   -0.03234860673546791,
   0.03299522027373314,
   -0.027164360508322716,
   0.03278419375419617,
   -0.03927069902420044,
   0.03773641958832741,
   0.06301204115152359,
   -0.018972747027873993,
   0.026979122310876846,
   0.02185557223856449,
   -0.04091086983680725,
   0.008678562939167023,
   0.03322387486696243,
   0.03242996335029602,
   0.03830456733703613,
   -0.009587320